In [1]:
import pandas as pd

file_path = '../data/machineLearningRating_v3.txt'
df = pd.read_csv(
    file_path,
    sep='|',               
    encoding='utf-8',        
    engine='python',
    on_bad_lines='skip'      
)
print("Shape:", df.shape)
df.head(3)



Shape: (1000098, 52)


,UnderwrittenCoverID,PolicyID,TransactionMonth,IsVATRegistered,Citizenship,LegalType,Title,Language,Bank,AccountType,...,ExcessSelected,CoverCategory,CoverType,CoverGroup,Section,Product,StatutoryClass,StatutoryRiskType,TotalPremium,TotalClaims
0,145249,12827,2015-03-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0
1,145249,12827,2015-05-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0
2,145249,12827,2015-07-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0


In [ ]:
df.info(memory_usage='deep')
import numpy as np

# Quick type check
df.info()

# Missing values by column
missing_summary = df.isnull().sum().sort_values(ascending=False)
missing_pct = (missing_summary / len(df)) * 100
missing_df = pd.DataFrame({
    'missing_count': missing_summary,
    'missing_percent': missing_pct
})
print(missing_df[missing_df.missing_count > 0].head(10))  # Top missing columns

# Detect mixed types (object columns that should be numeric, etc.)
mixed_type_cols = [col for col in df.columns if df[col].apply(type).nunique() > 1]
print("Columns with mixed types:", mixed_type_cols)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000098 entries, 0 to 1000097
Data columns (total 52 columns):
 #   Column                    Non-Null Count    Dtype  
---  ------                    --------------    -----  
 0   UnderwrittenCoverID       1000098 non-null  int64  
 1   PolicyID                  1000098 non-null  int64  
 2   TransactionMonth          1000098 non-null  object 
 3   IsVATRegistered           1000098 non-null  bool   
 4   Citizenship               1000098 non-null  object 
 5   LegalType                 1000098 non-null  object 
 6   Title                     1000098 non-null  object 
 7   Language                  1000098 non-null  object 
 8   Bank                      854137 non-null   object 
 9   AccountType               959866 non-null   object 
 10  MaritalStatus             991839 non-null   object 
 11  Gender                    990562 non-null   object 
 12  Country                   1000098 non-null  object 
 13  Province                  1

In [ ]:
# 1. Check for fully duplicated rows
duplicate_rows = df[df.duplicated()]
print(f"Fully duplicated rows: {duplicate_rows.shape[0]}")


Fully duplicated rows: 0


In [ ]:
# Define placeholder strings often used to mean "missing"
empty_like_values = ['', ' ', 'nan', 'NaN', 'NULL', '-', '.', 'N/A']

# Count suspiciously empty-looking entries in each column
suspicious_counts = {}

for col in df.columns:
    if df[col].dtype == object or pd.api.types.is_string_dtype(df[col]):
        # Convert to string, strip whitespace, and check if value is in placeholder list
        count = df[col].astype(str).str.strip().isin(empty_like_values).sum()
        if count > 0:
            suspicious_counts[col] = count

# Display all columns with suspicious values
suspicious_df = pd.DataFrame(list(suspicious_counts.items()), columns=['Column', 'SuspiciousEmptyCount'])
suspicious_df = suspicious_df.sort_values(by='SuspiciousEmptyCount', ascending=False)

print(suspicious_df)

                Column  SuspiciousEmptyCount
15         CrossBorder                999400
0          Citizenship                895210
14           Converted                641901
12          WrittenOff                641901
13             Rebuilt                641901
11          NewVehicle                153295
1                 Bank                145961
2          AccountType                 40232
4               Gender                  9536
3        MaritalStatus                  8259
5          VehicleType                   552
6                 make                   552
7                Model                   552
8             bodytype                   552
9     VehicleIntroDate                   552
10  CapitalOutstanding                     2


""" 1. CrossBorder
- Indicates whether a vehicle or policyholder has been involved in cross-border activities (e.g., import/export, usage across countries).

Insurance Value:

 — Can be important for fraud detection, risk assessment, or regulatory compliance.

However — If it's ~99.94% missing, it may not be reliably useful unless i can recover that data from another source.

Verdict:  High value in theory, but with 99.9% missing, likely unusable.

 2. Citizenship
- Likely the citizenship status of the customer.

Insurance Value:

Useful for KYC (Know Your Customer), risk scoring, or eligibility (e.g., certain citizenships may have different coverage rules).

Can also correlate with fraud checks, travel patterns, or international claims.

Verdict:  Potentially valuable for regulatory and risk purposes. But with ~89% missing, it's limited in its utility unless cleaned or enhanced.

 3. Converted
- Might indicate whether a quote or lead was converted to a policy or converted vehicle type (e.g., modified car).

Insurance Value:

If it's about sales conversion: very useful for sales funnel analysis.

If it's about vehicle modification: important for underwriting and pricing risk.

Verdict:  Useful, depending on exact meaning. But 64% missing — keep only if meaning is confirmed and missingness is explainable.

 4. WrittenOff
--  Indicates if the vehicle was declared a total loss (“written off”) after damage.

Insurance Value:

Very important for claims history and underwriting.

Vehicles previously written off may be ineligible or have special terms.

Useful for fraud detection if someone tries to insure already written-off vehicles.

-  High business value for underwriting and claims analysis — worth cleaning or sourcing better.

 5. Rebuilt
- Indicates if a vehicle was rebuilt after being written off (common in salvage or refurbished vehicle cases).

Insurance Value:

Critical for vehicle valuation and coverage terms.

Rebuilt vehicles often require special handling or premiums.

Verdict:  Important for pricing and risk — may justify effort to recover or clean data."""

In [ ]:
#lets drop high empty columns 
cols_to_drop = ['CrossBorder', 'Citizenship']
df.drop(columns=cols_to_drop, inplace=True)
print("Dropped columns:", cols_to_drop)
# Check remaining columns after dropping
print("Remaining columns after drop:", df.columns.tolist())

Dropped columns: ['CrossBorder', 'Citizenship']
Remaining columns after drop: ['UnderwrittenCoverID', 'PolicyID', 'TransactionMonth', 'IsVATRegistered', 'LegalType', 'Title', 'Language', 'Bank', 'AccountType', 'MaritalStatus', 'Gender', 'Country', 'Province', 'PostalCode', 'MainCrestaZone', 'SubCrestaZone', 'ItemType', 'mmcode', 'VehicleType', 'RegistrationYear', 'make', 'Model', 'Cylinders', 'cubiccapacity', 'kilowatts', 'bodytype', 'NumberOfDoors', 'VehicleIntroDate', 'CustomValueEstimate', 'AlarmImmobiliser', 'TrackingDevice', 'CapitalOutstanding', 'NewVehicle', 'WrittenOff', 'Rebuilt', 'Converted', 'NumberOfVehiclesInFleet', 'SumInsured', 'TermFrequency', 'CalculatedPremiumPerTerm', 'ExcessSelected', 'CoverCategory', 'CoverType', 'CoverGroup', 'Section', 'Product', 'StatutoryClass', 'StatutoryRiskType', 'TotalPremium', 'TotalClaims']


In [ ]:
#retian risk flag for converted, writen off, rebuilt,
#imputing missing vlaues as "unknown"
for col in ['Converted', 'WrittenOff', 'Rebuilt']:
    df[col] = df[col].fillna('Unknown')  # or False
print("Imputed missing values for risk flags:", ['Converted', 'WrittenOff', 'Rebuilt'])
# Check if any of these columns still have missing values
print("Missing values after imputation:")
print(df[['Converted', 'WrittenOff', 'Rebuilt']].isnull().sum())

Imputed missing values for risk flags: ['Converted', 'WrittenOff', 'Rebuilt']
Missing values after imputation:
Converted     0
WrittenOff    0
Rebuilt       0
dtype: int64


In [ ]:
df.head(10)

,UnderwrittenCoverID,PolicyID,TransactionMonth,IsVATRegistered,LegalType,Title,Language,Bank,AccountType,MaritalStatus,Gender,Country,Province,PostalCode,MainCrestaZone,SubCrestaZone,ItemType,mmcode,VehicleType,RegistrationYear,make,Model,Cylinders,cubiccapacity,kilowatts,bodytype,NumberOfDoors,VehicleIntroDate,CustomValueEstimate,AlarmImmobiliser,TrackingDevice,CapitalOutstanding,NewVehicle,WrittenOff,Rebuilt,Converted,NumberOfVehiclesInFleet,SumInsured,TermFrequency,CalculatedPremiumPerTerm,ExcessSelected,CoverCategory,CoverType,CoverGroup,Section,Product,StatutoryClass,StatutoryRiskType,TotalPremium,TotalClaims
0,145249,12827,2015-03-01 00:00:00,True,Close Corporation,Mr,English,First National Bank,Current account,Not specified,Not specified,South Africa,Gauteng,1459,Rand East,Rand East,Mobility - Motor,44069150.0,Passenger Vehicle,2004,MERCEDES-BENZ,E 240,6.0,2597.0,130.0,S/D,4.0,6/2002,119300.0,Yes,No,119300,More than 6 months,Unknown,Unknown,Unknown,NaN,0.01,Monthly,25.0000,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0
1,145249,12827,2015-05-01 00:00:00,True,Close Corporation,Mr,English,First National Bank,Current account,Not specified,Not specified,South Africa,Gauteng,1459,Rand East,Rand East,Mobility - Motor,44069150.0,Passenger Vehicle,2004,MERCEDES-BENZ,E 240,6.0,2597.0,130.0,S/D,4.0,6/2002,119300.0,Yes,No,119300,More than 6 months,Unknown,Unknown,Unknown,NaN,0.01,Monthly,25.0000,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0
2,145249,12827,2015-07-01 00:00:00,True,Close Corporation,Mr,English,First National Bank,Current account,Not specified,Not specified,South Africa,Gauteng,1459,Rand East,Rand East,Mobility - Motor,44069150.0,Passenger Vehicle,2004,MERCEDES-BENZ,E 240,6.0,2597.0,130.0,S/D,4.0,6/2002,119300.0,Yes,No,119300,More than 6 months,Unknown,Unknown,Unknown,NaN,0.01,Monthly,25.0000,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0
3,145255,12827,2015-05-01 00:00:00,True,Close Corporation,Mr,English,First National Bank,Current account,Not specified,Not specified,South Africa,Gauteng,1459,Rand East,Rand East,Mobility - Motor,44069150.0,Passenger Vehicle,2004,MERCEDES-BENZ,E 240,6.0,2597.0,130.0,S/D,4.0,6/2002,119300.0,Yes,No,119300,More than 6 months,Unknown,Unknown,Unknown,NaN,119300.00,Monthly,584.6468,Mobility - Metered Taxis - R2000,Own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,512.848070,0.0
4,145255,12827,2015-07-01 00:00:00,True,Close Corporation,Mr,English,First National Bank,Current account,Not specified,Not specified,South Africa,Gauteng,1459,Rand East,Rand East,Mobility - Motor,44069150.0,Passenger Vehicle,2004,MERCEDES-BENZ,E 240,6.0,2597.0,130.0,S/D,4.0,6/2002,119300.0,Yes,No,119300,More than 6 months,Unknown,Unknown,Unknown,NaN,119300.00,Monthly,584.6468,Mobility - Metered Taxis - R2000,Own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0
5,145247,12827,2015-01-01 00:00:00,True,Close Corporation,Mr,English,First National Bank,Current account,Not specified,Not specified,South Africa,Gauteng,1459,Rand East,Rand East,Mobility - Motor,44069150.0,Passenger Vehicle,2004,MERCEDES-BENZ,E 240,6.0,2597.0,130.0,S/D,4.0,6/2002,119300.0,Yes,No,119300,More than 6 months,Unknown,Unknown,Unknown,NaN,500000.00,Monthly,57.5412,No excess,Third Party,Third Party,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,3.256435,0.0
6,145247,12827,2015-04-01 00:00:00,True,Close Corporation,Mr,English,First National Bank,Current account,Not specified,Not specified,South Africa,Gauteng,1459,Rand East,Rand Eas

In [ ]:
# group or impute moderate missing columns
# for columns like bank, account type, gender, matital status
df['Bank'] = df['Bank'].fillna('Unknown')
df['Gender'] = df['Gender'].fillna('Unknown')
df['MaritalStatus'] = df['MaritalStatus'].fillna('Unknown')
df['AccountType'] = df['AccountType'].fillna('Unknown')


In [ ]:
df.head(10)

,UnderwrittenCoverID,PolicyID,TransactionMonth,IsVATRegistered,LegalType,Title,Language,Bank,AccountType,MaritalStatus,Gender,Country,Province,PostalCode,MainCrestaZone,SubCrestaZone,ItemType,mmcode,VehicleType,RegistrationYear,make,Model,Cylinders,cubiccapacity,kilowatts,bodytype,NumberOfDoors,VehicleIntroDate,CustomValueEstimate,AlarmImmobiliser,TrackingDevice,CapitalOutstanding,NewVehicle,WrittenOff,Rebuilt,Converted,NumberOfVehiclesInFleet,SumInsured,TermFrequency,CalculatedPremiumPerTerm,ExcessSelected,CoverCategory,CoverType,CoverGroup,Section,Product,StatutoryClass,StatutoryRiskType,TotalPremium,TotalClaims
0,145249,12827,2015-03-01 00:00:00,True,Close Corporation,Mr,English,First National Bank,Current account,Not specified,Not specified,South Africa,Gauteng,1459,Rand East,Rand East,Mobility - Motor,44069150.0,Passenger Vehicle,2004,MERCEDES-BENZ,E 240,6.0,2597.0,130.0,S/D,4.0,6/2002,119300.0,Yes,No,119300,More than 6 months,Unknown,Unknown,Unknown,NaN,0.01,Monthly,25.0000,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0
1,145249,12827,2015-05-01 00:00:00,True,Close Corporation,Mr,English,First National Bank,Current account,Not specified,Not specified,South Africa,Gauteng,1459,Rand East,Rand East,Mobility - Motor,44069150.0,Passenger Vehicle,2004,MERCEDES-BENZ,E 240,6.0,2597.0,130.0,S/D,4.0,6/2002,119300.0,Yes,No,119300,More than 6 months,Unknown,Unknown,Unknown,NaN,0.01,Monthly,25.0000,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0
2,145249,12827,2015-07-01 00:00:00,True,Close Corporation,Mr,English,First National Bank,Current account,Not specified,Not specified,South Africa,Gauteng,1459,Rand East,Rand East,Mobility - Motor,44069150.0,Passenger Vehicle,2004,MERCEDES-BENZ,E 240,6.0,2597.0,130.0,S/D,4.0,6/2002,119300.0,Yes,No,119300,More than 6 months,Unknown,Unknown,Unknown,NaN,0.01,Monthly,25.0000,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0
3,145255,12827,2015-05-01 00:00:00,True,Close Corporation,Mr,English,First National Bank,Current account,Not specified,Not specified,South Africa,Gauteng,1459,Rand East,Rand East,Mobility - Motor,44069150.0,Passenger Vehicle,2004,MERCEDES-BENZ,E 240,6.0,2597.0,130.0,S/D,4.0,6/2002,119300.0,Yes,No,119300,More than 6 months,Unknown,Unknown,Unknown,NaN,119300.00,Monthly,584.6468,Mobility - Metered Taxis - R2000,Own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,512.848070,0.0
4,145255,12827,2015-07-01 00:00:00,True,Close Corporation,Mr,English,First National Bank,Current account,Not specified,Not specified,South Africa,Gauteng,1459,Rand East,Rand East,Mobility - Motor,44069150.0,Passenger Vehicle,2004,MERCEDES-BENZ,E 240,6.0,2597.0,130.0,S/D,4.0,6/2002,119300.0,Yes,No,119300,More than 6 months,Unknown,Unknown,Unknown,NaN,119300.00,Monthly,584.6468,Mobility - Metered Taxis - R2000,Own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0
5,145247,12827,2015-01-01 00:00:00,True,Close Corporation,Mr,English,First National Bank,Current account,Not specified,Not specified,South Africa,Gauteng,1459,Rand East,Rand East,Mobility - Motor,44069150.0,Passenger Vehicle,2004,MERCEDES-BENZ,E 240,6.0,2597.0,130.0,S/D,4.0,6/2002,119300.0,Yes,No,119300,More than 6 months,Unknown,Unknown,Unknown,NaN,500000.00,Monthly,57.5412,No excess,Third Party,Third Party,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,3.256435,0.0
6,145247,12827,2015-04-01 00:00:00,True,Close Corporation,Mr,English,First National Bank,Current account,Not specified,Not specified,South Africa,Gauteng,1459,Rand East,Rand Eas

In [ ]:
#fix minor missing columns for vihicles type, make, model , bodytype and vehicle introdate
df.dropna(subset=['VehicleType', 'make', 'Model','VehicleIntroDate','bodytype','CapitalOutstanding'], inplace=True)
# Check if any of these columns still have missing values
print("Missing values after dropping rows with minor missing columns:")
print(df[['VehicleType', 'make', 'Model', 'VehicleIntroDate', 'bodytype','CapitalOutstanding']].isnull().sum())

Missing values after dropping rows with minor missing columns:
VehicleType           0
make                  0
Model                 0
VehicleIntroDate      0
bodytype              0
CapitalOutstanding    0
dtype: int64


In [ ]:
# now lets check any missing values in any columns 
missing_summary = df.isnull().sum().sort_values(ascending=False)
missing_pct = (missing_summary / len(df)) * 100
missing_df = pd.DataFrame({
    'missing_count': missing_summary,
    'missing_percent': missing_pct
})
print("Final missing values summary:")
print(missing_df[missing_df.missing_count > 0].head(10))  # Top missing columns after cleaning

Final missing values summary:
                         missing_count  missing_percent
NumberOfVehiclesInFleet         999544       100.000000
CustomValueEstimate             779088        77.944343
NewVehicle                      153295        15.336493


In [ ]:
#drope customeValueE    stimate as it has 787% missing values
df.drop(columns=['CustomValueEstimate'], inplace=True)
print("Dropped column: CustomValueEstimate due to high missing values.")
# drop numberof vehicles as it has 100% missing values
df.drop(columns=['NumberOfVehiclesInFleet'], inplace=True)
print("Dropped column: NumberOfVehiclesInFleet due to 100% missing values.")


Dropped column: CustomValueEstimate due to high missing values.
Dropped column: NumberOfVehiclesInFleet due to 100% missing values.


In [ ]:
# Fill missing NewVehicle values with 'Unknown'
df['NewVehicle'] = df['NewVehicle'].fillna('Unknown')
# Check if any of these columns still have missing values
print("Missing values after filling NewVehicle:")
print(df['NewVehicle'].isnull().sum())
# Check for any remaining missing values
print("Final missing values summary after all cleaning:")
missing_summary = df.isnull().sum().sort_values(ascending=False)
missing_pct = (missing_summary / len(df)) * 100
missing_df = pd.DataFrame({
    'missing_count': missing_summary,
    'missing_percent': missing_pct
})
print(missing_df[missing_df.missing_count > 0].head(10))  # Top missing columns after all cleaning

Missing values after filling NewVehicle:
0
Final missing values summary after all cleaning:
Empty DataFrame
Columns: [missing_count, missing_percent]
Index: []


In [ ]:

# Check remaining columns after dropping
print("Remaining columns after final drop:", df.columns.tolist())

Remaining columns after final drop: ['UnderwrittenCoverID', 'PolicyID', 'TransactionMonth', 'IsVATRegistered', 'LegalType', 'Title', 'Language', 'Bank', 'AccountType', 'MaritalStatus', 'Gender', 'Country', 'Province', 'PostalCode', 'MainCrestaZone', 'SubCrestaZone', 'ItemType', 'mmcode', 'VehicleType', 'RegistrationYear', 'make', 'Model', 'Cylinders', 'cubiccapacity', 'kilowatts', 'bodytype', 'NumberOfDoors', 'VehicleIntroDate', 'AlarmImmobiliser', 'TrackingDevice', 'CapitalOutstanding', 'NewVehicle', 'WrittenOff', 'Rebuilt', 'Converted', 'SumInsured', 'TermFrequency', 'CalculatedPremiumPerTerm', 'ExcessSelected', 'CoverCategory', 'CoverType', 'CoverGroup', 'Section', 'Product', 'StatutoryClass', 'StatutoryRiskType', 'TotalPremium', 'TotalClaims']


In [ ]:
# now lets check data types
# Check data types of all columns
print("Data types of all columns:")
df.dtypes.sort_values()


Data types of all columns:


IsVATRegistered                bool
UnderwrittenCoverID           int64
PolicyID                      int64
RegistrationYear              int64
PostalCode                    int64
CalculatedPremiumPerTerm    float64
SumInsured                  float64
NumberOfDoors               float64
kilowatts                   float64
TotalPremium                float64
Cylinders                   float64
mmcode                      float64
cubiccapacity               float64
TotalClaims                 float64
WrittenOff                   object
Rebuilt                      object
Converted                    object
LegalType                    object
TermFrequency                object
TransactionMonth             object
NewVehicle                   object
ExcessSelected               object
CoverType                    object
CoverGroup                   object
Section                      object
Product                      object
StatutoryClass               object
StatutoryRiskType           

| Column               | Problem                         | Expected Type    | Reason/Fix                              |
| -------------------- | ------------------------------- | ---------------- | --------------------------------------- |
| `TransactionMonth`   |  is `object` (string)          | `datetime64[ns]` | Time-based analysis, sorting, filtering |
| `VehicleIntroDate`   |  is `object`                   | `datetime64[ns]` | Needed to compute vehicle age, trends   |
| `NewVehicle`         |  is `object`                   | `bool`           | Logical column (Yes/No)                 |
| `WrittenOff`         |  is `object`                   | `bool`           | Logical flag                            |
| `Rebuilt`            |  is `object`                   | `bool`           | Logical flag                            |
| `Converted`          |  is `object`                   | `bool`           | Logical flag                            |
| `CapitalOutstanding` |  is `object` (should be float) | `float64`        | Financial amount                        |


"""Already Correct:
Booleans: IsVATRegistered 

Numerical: All key numeric columns like TotalPremium, SumInsured, etc. are correctly float64 

Identifiers: PolicyID, UnderwrittenCoverID are int64 """

In [ ]:
#fix dates
df['TransactionMonth'] = pd.to_datetime(df['TransactionMonth'], errors='coerce')
df['VehicleIntroDate'] = pd.to_datetime(df['VehicleIntroDate'], errors='coerce')

# Check again
print(df[['TransactionMonth', 'VehicleIntroDate']].dtypes)


TransactionMonth    datetime64[ns]
VehicleIntroDate    datetime64[ns]
dtype: object


/tmp/ipykernel_56564/542130726.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['VehicleIntroDate'] = pd.to_datetime(df['VehicleIntroDate'], errors='coerce')


In [ ]:
#lets convert binary columns to boolean
binary_cols = ['NewVehicle', 'WrittenOff', 'Rebuilt', 'Converted']

for col in binary_cols:
    df[col] = df[col].astype(str).str.strip().str.lower()
    df[col] = df[col].replace({'yes': True, 'no': False, 'unknown': False, '': False, 'nan': False})
    df[col] = df[col].replace('nan', False).fillna(False)
    df[col] = df[col].astype(bool)

print("Data types after conversion:")
print(df[binary_cols].dtypes)


/tmp/ipykernel_56564/232422190.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].replace({'yes': True, 'no': False, 'unknown': False, '': False, 'nan': False})
/tmp/ipykernel_56564/232422190.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].replace({'yes': True, 'no': False, 'unknown': False, '': False, 'nan': False})


Data types after conversion:
NewVehicle    bool
WrittenOff    bool
Rebuilt       bool
Converted     bool
dtype: object


/tmp/ipykernel_56564/232422190.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].replace({'yes': True, 'no': False, 'unknown': False, '': False, 'nan': False})


In [ ]:
# capital outstanding should be float
df['CapitalOutstanding'] = (
    df['CapitalOutstanding']
    .astype(str)
    .str.replace(',', '')
    .str.extract(r'(\d+\.?\d*)')[0]  # Extract just the number part
)

df['CapitalOutstanding'] = pd.to_numeric(df['CapitalOutstanding'], errors='coerce')
# Check if conversion was successful
print("Data types after converting CapitalOutstanding:")
print(df['CapitalOutstanding'].dtype)
# Check final data types
print("Final data types of all columns:")
print(df.dtypes.sort_values())

Data types after converting CapitalOutstanding:
float64
Final data types of all columns:
NewVehicle                            bool
PostalCode                           int64
VehicleIntroDate            datetime64[ns]
IsVATRegistered                       bool
WrittenOff                            bool
Converted                             bool
Rebuilt                               bool
UnderwrittenCoverID                  int64
PolicyID                             int64
RegistrationYear                     int64
Cylinders                          float64
CapitalOutstanding                 float64
CalculatedPremiumPerTerm           float64
NumberOfDoors                      float64
kilowatts                          float64
TotalPremium                       float64
SumInsured                         float64
cubiccapacity                      float64
TotalClaims                        float64
TransactionMonth            datetime64[ns]
mmcode                             float64
Statutor

In [ ]:
df.shape

(999544, 48)

In [ ]:
# since we dont have unique id for all rows 
df['RecordID'] = df.index
print(df.shape)  # Should print (1000098, 53)


(999544, 49)


In [ ]:
#check if record is unique
# so i have give just record id column that uniquly idntifies each raws
assert df['RecordID'].is_unique


In [ ]:
#lets put upfront the record id column
cols = ['RecordID'] + [col for col in df.columns if col != 'RecordID']
df = df[cols]


In [ ]:
df.head(10)

,RecordID,UnderwrittenCoverID,PolicyID,TransactionMonth,IsVATRegistered,LegalType,Title,Language,Bank,AccountType,MaritalStatus,Gender,Country,Province,PostalCode,MainCrestaZone,SubCrestaZone,ItemType,mmcode,VehicleType,RegistrationYear,make,Model,Cylinders,cubiccapacity,kilowatts,bodytype,NumberOfDoors,VehicleIntroDate,AlarmImmobiliser,TrackingDevice,CapitalOutstanding,NewVehicle,WrittenOff,Rebuilt,Converted,SumInsured,TermFrequency,CalculatedPremiumPerTerm,ExcessSelected,CoverCategory,CoverType,CoverGroup,Section,Product,StatutoryClass,StatutoryRiskType,TotalPremium,TotalClaims
0,0,145249,12827,2015-03-01,True,Close Corporation,Mr,English,First National Bank,Current account,Not specified,Not specified,South Africa,Gauteng,1459,Rand East,Rand East,Mobility - Motor,44069150.0,Passenger Vehicle,2004,MERCEDES-BENZ,E 240,6.0,2597.0,130.0,S/D,4.0,2002-06-01,Yes,No,119300.0,True,False,False,False,0.01,Monthly,25.0000,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0
1,1,145249,12827,2015-05-01,True,Close Corporation,Mr,English,First National Bank,Current account,Not specified,Not specified,South Africa,Gauteng,1459,Rand East,Rand East,Mobility - Motor,44069150.0,Passenger Vehicle,2004,MERCEDES-BENZ,E 240,6.0,2597.0,130.0,S/D,4.0,2002-06-01,Yes,No,119300.0,True,False,False,False,0.01,Monthly,25.0000,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0
2,2,145249,12827,2015-07-01,True,Close Corporation,Mr,English,First National Bank,Current account,Not specified,Not specified,South Africa,Gauteng,1459,Rand East,Rand East,Mobility - Motor,44069150.0,Passenger Vehicle,2004,MERCEDES-BENZ,E 240,6.0,2597.0,130.0,S/D,4.0,2002-06-01,Yes,No,119300.0,True,False,False,False,0.01,Monthly,25.0000,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0
3,3,145255,12827,2015-05-01,True,Close Corporation,Mr,English,First National Bank,Current account,Not specified,Not specified,South Africa,Gauteng,1459,Rand East,Rand East,Mobility - Motor,44069150.0,Passenger Vehicle,2004,MERCEDES-BENZ,E 240,6.0,2597.0,130.0,S/D,4.0,2002-06-01,Yes,No,119300.0,True,False,False,False,119300.00,Monthly,584.6468,Mobility - Metered Taxis - R2000,Own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,512.848070,0.0
4,4,145255,12827,2015-07-01,True,Close Corporation,Mr,English,First National Bank,Current account,Not specified,Not specified,South Africa,Gauteng,1459,Rand East,Rand East,Mobility - Motor,44069150.0,Passenger Vehicle,2004,MERCEDES-BENZ,E 240,6.0,2597.0,130.0,S/D,4.0,2002-06-01,Yes,No,119300.0,True,False,False,False,119300.00,Monthly,584.6468,Mobility - Metered Taxis - R2000,Own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0
5,5,145247,12827,2015-01-01,True,Close Corporation,Mr,English,First National Bank,Current account,Not specified,Not specified,South Africa,Gauteng,1459,Rand East,Rand East,Mobility - Motor,44069150.0,Passenger Vehicle,2004,MERCEDES-BENZ,E 240,6.0,2597.0,130.0,S/D,4.0,2002-06-01,Yes,No,119300.0,True,False,False,False,500000.00,Monthly,57.5412,No excess,Third Party,Third Party,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,3.256435,0.0
6,6,145247,12827,2015-04-01,True,Close Corporation,Mr,English,First National Bank,Current account,Not specified,Not specified,South Africa,Gauteng,1459,Rand East,Rand East,Mobility - Motor,44069150.0,Passenger Vehicle,2004,MERCEDES-BENZ,E 240,6.0,2597.0,130.0,S/D,4.0,2002-06-01,Yes,No,119300.0,True,False,False,False,500000.00,Monthly,57.5412,No excess,Third Party,Third Party,Comprehensive - Taxi,Motor Comprehensi

In [ ]:
# Set pandas option to display all columns
pd.set_option('display.max_columns', None)

# Now display your DataFrame head to see all columns
print(df.head())


   RecordID  UnderwrittenCoverID  PolicyID TransactionMonth  IsVATRegistered  \
0         0               145249     12827       2015-03-01             True   
1         1               145249     12827       2015-05-01             True   
2         2               145249     12827       2015-07-01             True   
3         3               145255     12827       2015-05-01             True   
4         4               145255     12827       2015-07-01             True   

           LegalType Title Language                 Bank      AccountType  \
0  Close Corporation    Mr  English  First National Bank  Current account   
1  Close Corporation    Mr  English  First National Bank  Current account   
2  Close Corporation    Mr  English  First National Bank  Current account   
3  Close Corporation    Mr  English  First National Bank  Current account   
4  Close Corporation    Mr  English  First National Bank  Current account   

   MaritalStatus         Gender       Country Province  

In [ ]:
# define catagorical columns to check
cat_cols = [
    'Gender', 'MaritalStatus', 'Province', 'VehicleType', 'Title',
    'CoverType', 'CoverCategory', 'NewVehicle', 'WrittenOff', 'Rebuilt',
    'Converted', 'Bank', 'AccountType', 'Language'
]


In [ ]:
# lets view unique values in categorical columns
for col in cat_cols:
    print(f"\n {col} ")
    print(df[col].value_counts(dropna=False))



 Gender 
Gender
Not specified    940436
Male              42817
Unknown            9536
Female             6755
Name: count, dtype: int64

 MaritalStatus 
MaritalStatus
Not specified    985654
Unknown            8259
Single             4254
Married            1377
Name: count, dtype: int64

 Province 
Province
Gauteng          393625
Western Cape     170562
KwaZulu-Natal    169781
North West       143287
Mpumalanga        52638
Eastern Cape      30336
Limpopo           24836
Free State         8099
Northern Cape      6380
Name: count, dtype: int64

 VehicleType 
VehicleType
Passenger Vehicle    933596
Medium Commercial     53985
Heavy Commercial       7401
Light Commercial       3897
Bus                     665
Name: count, dtype: int64

 Title 
Title
Mr      933002
Mrs      45849
Ms       13269
Miss      6614
Dr         810
Name: count, dtype: int64

 CoverType 
CoverType
Own Damage                                             104120
Passenger Liability                                

In [ ]:
# so based on above output 
"""| Issue                                | Solution                                  |
| ------------------------------------ | ----------------------------------------- |
| Majority is `"Not specified"` (940k) | Normalize `"Not specified"` → `'unknown'` |
| Mixed casing (`Male`, `Female`)      | Convert to lowercase                      |

"""
df['Gender'] = df['Gender'].str.strip().str.lower().replace({
    'not specified': 'unknown',
    '': 'unknown'
})

# same for marital status
df['MaritalStatus'] = df['MaritalStatus'].str.strip().str.lower().replace({
    'not specified': 'unknown',
    '': 'unknown'
})

In [ ]:

# province
#vehicle type both no need change 
# for title lets chenge to lower case all

""" Title
Titles are clean (Mr, Mrs, etc.)
 Optional: Normalize to lowercase:"""

df['Title'] = df['Title'].str.strip().str.lower()\
# cover type is clean


In [ ]:
""" 
CoverCategory

Needs cleanup:

Variants like "Own damage", "Own Damage.", "Own Damage"

Versions like "Windscreen (2015)", "Income Protector (2015)"

Strategy:

Normalize lowercase

Map noisy variants to clean categories"""

df['CoverCategory'] = df['CoverCategory'].str.strip().str.lower()

# Manual mapping for known noisy duplicates
df['CoverCategory'] = df['CoverCategory'].replace({
    'own damage.': 'own damage',
    'own damage': 'own damage',
    'windscreen (2015)': 'windscreen',
    'income protector (2015)': 'income protector',
    'basic excess waiver (2015)': 'basic excess waiver'
})

"""NewVehicle, WrittenOff, Rebuilt, Converted
Already cleaned — values are Boolean: True/False
No further action needed."""

'NewVehicle, WrittenOff, Rebuilt, Converted\nAlready cleaned — values are Boolean: True/False\nNo further action needed.'

In [ ]:
"""Bank
Slight variation between banks (FirstRand Bank, First National Bank)"""
# so lets normalize bank names to lowercase and remove extra spaces
# Optional cleanup
df['Bank'] = df['Bank'].str.strip().str.lower().replace({
    'firstrand bank': 'first national bank'
})


In [ ]:
"""AccountType
Mostly clean
Optional: normalize casing & spacing"""
df['AccountType'] = df['AccountType'].str.strip().str.lower()


In [ ]:
"""Language
Uniform — all "English"
Consider dropping it, as it adds no value:"""

df.drop(columns=['Language'], inplace=True)
print("Dropped column: Language as it adds no value.")
# Check remaining columns after cleanup
print("Remaining columns after cleanup:", df.columns.tolist())

Dropped column: Language as it adds no value.
Remaining columns after cleanup: ['RecordID', 'UnderwrittenCoverID', 'PolicyID', 'TransactionMonth', 'IsVATRegistered', 'LegalType', 'Title', 'Bank', 'AccountType', 'MaritalStatus', 'Gender', 'Country', 'Province', 'PostalCode', 'MainCrestaZone', 'SubCrestaZone', 'ItemType', 'mmcode', 'VehicleType', 'RegistrationYear', 'make', 'Model', 'Cylinders', 'cubiccapacity', 'kilowatts', 'bodytype', 'NumberOfDoors', 'VehicleIntroDate', 'AlarmImmobiliser', 'TrackingDevice', 'CapitalOutstanding', 'NewVehicle', 'WrittenOff', 'Rebuilt', 'Converted', 'SumInsured', 'TermFrequency', 'CalculatedPremiumPerTerm', 'ExcessSelected', 'CoverCategory', 'CoverType', 'CoverGroup', 'Section', 'Product', 'StatutoryClass', 'StatutoryRiskType', 'TotalPremium', 'TotalClaims']


### outlier detecton and removal

In [ ]:
#basic numeric column description
# checking for min max and un usual values 
df[['TotalPremium', 'TotalClaims', 'SumInsured', 'CalculatedPremiumPerTerm',
    'CapitalOutstanding', 'Cylinders', 'kilowatts', 'cubiccapacity']].describe()


,TotalPremium,TotalClaims,SumInsured,CalculatedPremiumPerTerm,CapitalOutstanding,Cylinders,kilowatts,cubiccapacity
count,999544.000000,999544.000000,9.995440e+05,999544.000000,9.995440e+05,999544.000000,999544.000000,999544.000000
mean,61.372350,64.340199,6.038605e+05,116.161425,6.164425e+04,4.046642,97.207964,2466.743461
std,157.014098,2369.567085,1.507329e+06,220.576166,5.156820e+05,0.294020,19.393250,442.801023
min,-782.576754,-12002.412281,1.000000e-02,0.000000,0.000000e+00,0.000000,0.000000,0.000000
25%,0.000000,0.000000,5.000000e+03,3.225700,0.000000e+00,4.000000,75.000000,2237.000000
50%,2.179737,0.000000,7.500000e+03,8.436900,0.000000e+00,4.000000,111.000000,2694.000000
75%,21.929825,0.000000,2.500000e+05,90.000000,0.000000e+00,4.000000,111.000000,2694.000000
max,2253.507281,393092.105263,1.000000e+07,3051.821100,2.857000e+07,10.000000,309.000000,12880.000000


In [ ]:
# check for negative or zero premiums 
# as insurance premiums should never be negative or zero 
print("Zero TotalPremium:", (df['TotalPremium'] == 0).sum())
print("Negative TotalPremium:", (df['TotalPremium'] < 0).sum())


Zero TotalPremium: 381302
Negative TotalPremium: 286


In [ ]:
# sometimes in insurace data, premiums can be zero for certain policies (e.g., free trials, promotional offers)
"""Zero Premiums
These could be:

Waived premiums (e.g., loyalty rewards, free periods)

Transactions like endorsements or policy changes

Data issues (incomplete records)"""
#but
# Negative Premiums
# Negative premiums are likely data errors or misentries
"""Negative Premiums
These are not valid in normal scenarios.

Could mean:

Canceled policies

Refunds

Bad input

Should be removed or flagged.""" 

# lets remove negative premiums
df = df[df['TotalPremium'] >= 0]
print("Removed negative TotalPremium. Remaining rows:", df.shape[0])
# Check again for zero premiums
zero_premiums = (df['TotalPremium'] == 0).sum()
print("Remaining zero TotalPremium:", zero_premiums)


Removed negative TotalPremium. Remaining rows: 999258
Remaining zero TotalPremium: 381302


In [ ]:
# check for any negative premiums 
#check negative premiums again
negative_premiums = (df['TotalPremium'] < 0).sum()
print("Remaining negative TotalPremium:", negative_premiums)

Remaining negative TotalPremium: 0


In [ ]:
# lets g to claims > premiums ? (loss ratio >1 )
df['LossRatio'] = df['TotalClaims'] / df['TotalPremium']

# Count and inspect
print("Loss Ratio > 1:", (df['LossRatio'] > 1).sum())
print("Max Loss Ratio:", df['LossRatio'].max())


Loss Ratio > 1: 2764
Max Loss Ratio: inf


In [ ]:
# the output inf max loss ratio mean it is devide by 0 from zero premiums
# so just calculate loss rattio only for non zero values
import numpy as np

# Loss ratio: only when TotalPremium > 0
df['LossRatio'] = np.where(
    df['TotalPremium'] > 0,
    df['TotalClaims'] / df['TotalPremium'],
    np.nan  # Set to NaN when premium is zero
)



In [ ]:
#verify 
print("Loss Ratio null values (should match TotalPremium = 0):", df['LossRatio'].isnull().sum())
print("Any infinite values in Loss Ratio?", np.isinf(df['LossRatio']).sum())


Loss Ratio null values (should match TotalPremium = 0): 381302
Any infinite values in Loss Ratio? 0


In [ ]:
# drope loss ration column
df.drop(columns=['LossRatio'], inplace=True)
print("Dropped column: LossRatio as it was derived and not needed for further analysis.")

Dropped column: LossRatio as it was derived and not needed for further analysis.


In [ ]:
df.head(10)

,RecordID,UnderwrittenCoverID,PolicyID,TransactionMonth,IsVATRegistered,LegalType,Title,Bank,AccountType,MaritalStatus,Gender,Country,Province,PostalCode,MainCrestaZone,SubCrestaZone,ItemType,mmcode,VehicleType,RegistrationYear,make,Model,Cylinders,cubiccapacity,kilowatts,bodytype,NumberOfDoors,VehicleIntroDate,AlarmImmobiliser,TrackingDevice,CapitalOutstanding,NewVehicle,WrittenOff,Rebuilt,Converted,SumInsured,TermFrequency,CalculatedPremiumPerTerm,ExcessSelected,CoverCategory,CoverType,CoverGroup,Section,Product,StatutoryClass,StatutoryRiskType,TotalPremium,TotalClaims
0,0,145249,12827,2015-03-01,True,Close Corporation,mr,first national bank,current account,unknown,unknown,South Africa,Gauteng,1459,Rand East,Rand East,Mobility - Motor,44069150.0,Passenger Vehicle,2004,MERCEDES-BENZ,E 240,6.0,2597.0,130.0,S/D,4.0,2002-06-01,Yes,No,119300.0,True,False,False,False,0.01,Monthly,25.0000,Mobility - Windscreen,windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0
1,1,145249,12827,2015-05-01,True,Close Corporation,mr,first national bank,current account,unknown,unknown,South Africa,Gauteng,1459,Rand East,Rand East,Mobility - Motor,44069150.0,Passenger Vehicle,2004,MERCEDES-BENZ,E 240,6.0,2597.0,130.0,S/D,4.0,2002-06-01,Yes,No,119300.0,True,False,False,False,0.01,Monthly,25.0000,Mobility - Windscreen,windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0
2,2,145249,12827,2015-07-01,True,Close Corporation,mr,first national bank,current account,unknown,unknown,South Africa,Gauteng,1459,Rand East,Rand East,Mobility - Motor,44069150.0,Passenger Vehicle,2004,MERCEDES-BENZ,E 240,6.0,2597.0,130.0,S/D,4.0,2002-06-01,Yes,No,119300.0,True,False,False,False,0.01,Monthly,25.0000,Mobility - Windscreen,windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0
3,3,145255,12827,2015-05-01,True,Close Corporation,mr,first national bank,current account,unknown,unknown,South Africa,Gauteng,1459,Rand East,Rand East,Mobility - Motor,44069150.0,Passenger Vehicle,2004,MERCEDES-BENZ,E 240,6.0,2597.0,130.0,S/D,4.0,2002-06-01,Yes,No,119300.0,True,False,False,False,119300.00,Monthly,584.6468,Mobility - Metered Taxis - R2000,own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,512.848070,0.0
4,4,145255,12827,2015-07-01,True,Close Corporation,mr,first national bank,current account,unknown,unknown,South Africa,Gauteng,1459,Rand East,Rand East,Mobility - Motor,44069150.0,Passenger Vehicle,2004,MERCEDES-BENZ,E 240,6.0,2597.0,130.0,S/D,4.0,2002-06-01,Yes,No,119300.0,True,False,False,False,119300.00,Monthly,584.6468,Mobility - Metered Taxis - R2000,own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0
5,5,145247,12827,2015-01-01,True,Close Corporation,mr,first national bank,current account,unknown,unknown,South Africa,Gauteng,1459,Rand East,Rand East,Mobility - Motor,44069150.0,Passenger Vehicle,2004,MERCEDES-BENZ,E 240,6.0,2597.0,130.0,S/D,4.0,2002-06-01,Yes,No,119300.0,True,False,False,False,500000.00,Monthly,57.5412,No excess,third party,Third Party,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,3.256435,0.0
6,6,145247,12827,2015-04-01,True,Close Corporation,mr,first national bank,current account,unknown,unknown,South Africa,Gauteng,1459,Rand East,Rand East,Mobility - Motor,44069150.0,Passenger Vehicle,2004,MERCEDES-BENZ,E 240,6.0,2597.0,130.0,S/D,4.0,2002-06-01,Yes,No,119300.0,True,False,False,False,500000.00,Monthly,57.5412,No excess,third party,Third Party,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,50.474737,0.0
7,7,145247,12827,2015-06-01,True,Close Corporation,mr,first national bank,c

In [ ]:
# so since most of our datas in gender and martial status is unknown
#lets identify these rows from title column

#normazlize title column to lower case
df['Title'] = df['Title'].str.strip().str.lower()


In [ ]:
# Fill missing/unknown GENDER
def infer_gender(row):
    if row['Gender'] == 'unknown':
        if row['Title'] == 'mr':
            return 'male'
        elif row['Title'] in ['mrs', 'miss', 'ms']:
            return 'female'
    return row['Gender']

# Fill missing/unknown MARITAL STATUS
def infer_marital_status(row):
    if row['MaritalStatus'] == 'unknown':
        if row['Title'] == 'mr':
            return 'unknown'
        elif row['Title'] == 'mrs':
            return 'married'
        elif row['Title'] == 'miss':
            return 'single'
    return row['MaritalStatus']


In [ ]:
# apply the above inference functions
df['Gender'] = df.apply(infer_gender, axis=1)
df['MaritalStatus'] = df.apply(infer_marital_status, axis=1)


In [ ]:
# lets find out the unkonwn again 
print("Remaining unknown Genders:", (df['Gender'] == 'unknown').sum())
print("Remaining unknown Marital Status:", (df['MaritalStatus'] == 'unknown').sum())


Remaining unknown Genders: 810
Remaining unknown Marital Status: 941888


In [ ]:
# so lets drop un known gendeers and marital status 
# drope column Marital status as it has 99% unknown values
df.drop(columns=['MaritalStatus'], inplace=True)
print("Dropped column: MaritalStatus due to high unknown values.")


Dropped column: MaritalStatus due to high unknown values.


In [ ]:
# delete raws of gender that have unknown valuess
df = df[df['Gender'] != 'unknown']

In [ ]:
# i saw some problem with total claimm values 
# Count total rows
total_rows = len(df)

# Count categories
neg_count = (df["TotalClaims"] < 0).sum()
zero_count = (df["TotalClaims"] == 0).sum()
pos_count = (df["TotalClaims"] > 0).sum()

# Print counts and percentages
print(" Negative TotalClaims:", neg_count, f"({neg_count / total_rows:.2%})")
print(" Zero TotalClaims:    ", zero_count, f"({zero_count / total_rows:.2%})")
print(" Positive TotalClaims:", pos_count, f"({pos_count / total_rows:.2%})")



 Negative TotalClaims: 5 (0.00%)
 Zero TotalClaims:     995670 (99.72%)
 Positive TotalClaims: 2773 (0.28%)


In [ ]:
#drop negative total claims
df = df[df['TotalClaims'] >= 0]

In [ ]:
df.head(10)

,RecordID,UnderwrittenCoverID,PolicyID,TransactionMonth,IsVATRegistered,LegalType,Title,Bank,AccountType,Gender,Country,Province,PostalCode,MainCrestaZone,SubCrestaZone,ItemType,mmcode,VehicleType,RegistrationYear,make,Model,Cylinders,cubiccapacity,kilowatts,bodytype,NumberOfDoors,VehicleIntroDate,AlarmImmobiliser,TrackingDevice,CapitalOutstanding,NewVehicle,WrittenOff,Rebuilt,Converted,SumInsured,TermFrequency,CalculatedPremiumPerTerm,ExcessSelected,CoverCategory,CoverType,CoverGroup,Section,Product,StatutoryClass,StatutoryRiskType,TotalPremium,TotalClaims
0,0,145249,12827,2015-03-01,True,Close Corporation,mr,first national bank,current account,male,South Africa,Gauteng,1459,Rand East,Rand East,Mobility - Motor,44069150.0,Passenger Vehicle,2004,MERCEDES-BENZ,E 240,6.0,2597.0,130.0,S/D,4.0,2002-06-01,Yes,No,119300.0,True,False,False,False,0.01,Monthly,25.0000,Mobility - Windscreen,windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0
1,1,145249,12827,2015-05-01,True,Close Corporation,mr,first national bank,current account,male,South Africa,Gauteng,1459,Rand East,Rand East,Mobility - Motor,44069150.0,Passenger Vehicle,2004,MERCEDES-BENZ,E 240,6.0,2597.0,130.0,S/D,4.0,2002-06-01,Yes,No,119300.0,True,False,False,False,0.01,Monthly,25.0000,Mobility - Windscreen,windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0
2,2,145249,12827,2015-07-01,True,Close Corporation,mr,first national bank,current account,male,South Africa,Gauteng,1459,Rand East,Rand East,Mobility - Motor,44069150.0,Passenger Vehicle,2004,MERCEDES-BENZ,E 240,6.0,2597.0,130.0,S/D,4.0,2002-06-01,Yes,No,119300.0,True,False,False,False,0.01,Monthly,25.0000,Mobility - Windscreen,windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0
3,3,145255,12827,2015-05-01,True,Close Corporation,mr,first national bank,current account,male,South Africa,Gauteng,1459,Rand East,Rand East,Mobility - Motor,44069150.0,Passenger Vehicle,2004,MERCEDES-BENZ,E 240,6.0,2597.0,130.0,S/D,4.0,2002-06-01,Yes,No,119300.0,True,False,False,False,119300.00,Monthly,584.6468,Mobility - Metered Taxis - R2000,own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,512.848070,0.0
4,4,145255,12827,2015-07-01,True,Close Corporation,mr,first national bank,current account,male,South Africa,Gauteng,1459,Rand East,Rand East,Mobility - Motor,44069150.0,Passenger Vehicle,2004,MERCEDES-BENZ,E 240,6.0,2597.0,130.0,S/D,4.0,2002-06-01,Yes,No,119300.0,True,False,False,False,119300.00,Monthly,584.6468,Mobility - Metered Taxis - R2000,own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0
5,5,145247,12827,2015-01-01,True,Close Corporation,mr,first national bank,current account,male,South Africa,Gauteng,1459,Rand East,Rand East,Mobility - Motor,44069150.0,Passenger Vehicle,2004,MERCEDES-BENZ,E 240,6.0,2597.0,130.0,S/D,4.0,2002-06-01,Yes,No,119300.0,True,False,False,False,500000.00,Monthly,57.5412,No excess,third party,Third Party,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,3.256435,0.0
6,6,145247,12827,2015-04-01,True,Close Corporation,mr,first national bank,current account,male,South Africa,Gauteng,1459,Rand East,Rand East,Mobility - Motor,44069150.0,Passenger Vehicle,2004,MERCEDES-BENZ,E 240,6.0,2597.0,130.0,S/D,4.0,2002-06-01,Yes,No,119300.0,True,False,False,False,500000.00,Monthly,57.5412,No excess,third party,Third Party,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,50.474737,0.0
7,7,145247,12827,2015-06-01,True,Close Corporation,mr,first national bank,current account,male,South Africa,Gauteng,1459,Rand East,Rand East,Mobility - Motor,44069150

In [ ]:
df.to_csv('../data/cleaned_machineLearningRating.csv', index=False)
print("\n Cleaning complete. Cleaned data saved to: cleaned_machineLearningRating.csv")


 Cleaning complete. Cleaned data saved to: cleaned_machineLearningRating.csv
